# Testing

In [1]:
%cd ..

/Users/udeepa/Documents/UCL/Term 2/bio/proteios


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import pickle
import copy
import collections

from Bio import SeqIO

from proteios.preprocess import preprocess, split_data
from proteios.featurise import Featuriser
from proteios.cross_val import cross_validate
from proteios.results import make_row, get_conf_matrix

In [3]:
np.random.seed(0)

### Get Data

In [4]:
input_path = "input_data/"

In [5]:
classes = ["Cytosolic", "Secreted", "Mitochondrial", "Nuclear"]

# Get dictionaries for labels
label2index = {key:i for i, key in enumerate(classes)}
index2label = dict(zip(label2index.values(), label2index.keys())) 

# Get data
datasets = dict()
for label in classes:
    datasets[label] = list(SeqIO.parse(input_path+label+".fasta", "fasta"))
# Get test data
blind_test_x = list(SeqIO.parse(input_path+"blind_test.fasta", "fasta"))

# Get number of examples in each category
counts = {key:len(val) for i, (key,val) in enumerate(datasets.items())}
counts["total"] = sum([len(sublist) for keys, sublist in datasets.items()])
print(label2index)
print(counts)
print("Number of test data points:  ", len(blind_test_x))

{'Cytosolic': 0, 'Secreted': 1, 'Mitochondrial': 2, 'Nuclear': 3}
{'Cytosolic': 3004, 'Secreted': 1605, 'Mitochondrial': 1299, 'Nuclear': 3314, 'total': 9222}
Number of test data points:   20


In [6]:
# Preprocess the data
full_x, full_y = preprocess(datasets, 
                            label2index=label2index,
                            trim_outliers=True,
                            max_length=2000)
print()
blind_test_xs = preprocess(blind_test_x, 
                           trim_outliers=True,
                           max_length=2000)
# Split the data
train_x, train_y, test_x, test_y = split_data(full_x, 
                                              full_y,
                                              train_size=0.9)
print()
print("Distribution Full:  ", dict(collections.Counter(full_y)))
print()
print("Training data size: ", len(train_y))
print("Test data size:     ", len(test_y))
print("Distribution train: ", dict(collections.Counter(train_y)))
print("Distribution test:  ", dict(collections.Counter(test_y)))

Processing Cytosolic
Data removed:  80
Processing Secreted
Data removed:  19
Processing Mitochondrial
Data removed:  2
Processing Nuclear
Data removed:  66
Total Before:  9222
Total After:   9055

Processing tmp
Data removed:  0
Total Before:  20
Total After:   20

Distribution Full:   {3: 3248, 0: 2924, 2: 1297, 1: 1586}

Training data size:  8148
Test data size:      907
Distribution train:  {1: 1427, 2: 1167, 3: 2923, 0: 2631}
Distribution test:   {3: 325, 0: 293, 1: 159, 2: 130}


### Get Features

In [7]:
# Get dictionaries
dicts = dict()
dicts['kd'] = pickle.load(open("input_data/scales/kd.pickle","rb"))
dicts['flex'] = pickle.load(open("input_data/scales/flex.pickle","rb"))
dicts['hw'] = pickle.load(open("input_data/scales/hw.pickle","rb"))
dicts['em'] = pickle.load(open("input_data/scales/em.pickle","rb"))
dicts['ja'] = pickle.load(open("input_data/scales/ja.pickle","rb"))
dicts['diwv'] = pickle.load(open("input_data/scales/diwv.pickle","rb"))  

In [8]:
featuriser = Featuriser(full_x, dicts)
full_x  = featuriser.transform(full_x)
blind_test_x  = featuriser.transform(blind_test_xs)
train_x = featuriser.transform(train_x)
test_x  = featuriser.transform(test_x)

In [9]:
train_x.head()

,length,molecular_weight,isoelectric_point,aromaticity,instability_index,gravy,reduced,oxidised,helix,turn,...,last_50_M,last_50_N,last_50_P,last_50_Q,last_50_R,last_50_S,last_50_T,last_50_V,last_50_W,last_50_Y
0,-0.806657,-0.800262,-0.755535,0.242616,-0.007247,0.305557,-0.509198,-0.508647,0.225385,0.012791,...,0.854124,-0.162842,0.138739,-0.110699,-0.806124,1.403898,-0.862108,-0.952807,0.422678,1.064776
1,-1.098142,-1.107773,1.170813,-1.528243,1.152644,0.437546,-0.901343,-0.906719,-0.683724,-0.370318,...,-0.016389,-0.162842,-0.289219,1.401648,0.907851,-0.787258,0.793750,-0.426707,1.502267,-0.978975
2,0.204178,0.224480,-0.802786,-0.099287,0.501211,0.927303,-0.435840,-0.429789,1.107682,-1.160093,...,-0.886903,-1.227841,2.278529,0.897533,-0.806124,-0.056872,-0.862108,-0.426707,-0.656912,-0.297724
3,0.046402,0.014496,1.297457,-1.188117,0.861330,0.038925,-0.447164,-0.428812,-1.140829,0.386568,...,2.595151,0.369657,3.990361,-0.614815,-0.806124,-0.422065,1.345703,0.099393,0.422678,-0.978975
4,0.899461,0.985599,-0.890165,1.067473,-0.433540,0.389393,0.506489,0.504784,1.612466,-0.363544,...,0.854124,-0.695342,-1.145135,-0.110699,0.907851,-0.787258,-0.862108,1.151592,0.422678,0.383526


In [10]:
for label in list(full_x.columns):
    print(label)
    print(full_x[label].mean())

length
-8.531122921415614e-17
molecular_weight
-4.4375816598849574e-16
isoelectric_point
-1.2866571419565315e-16
aromaticity
-5.608863885643557e-15
instability_index
9.291542932318455e-16
gravy
-5.442459171229282e-15
reduced
-9.717976469551618e-17
oxidised
5.640006530398366e-18
helix
4.935054757633617e-15
turn
1.8224332405768527e-15
sheet
8.634975480522146e-15
charge_at_ph1
3.86460603993153e-15
charge_at_ph7
6.172128885657689e-17
charge_at_ph12
-5.974728657093747e-16
hydrophobicity
-2.651955592360835e-15
flexibility
1.5506673189505305e-13
hydrophilicity
9.183892372890417e-16
surface_accessibility
-3.666092523122022e-14
janin
2.369428047839162e-15
first50_hydrophobicity
1.882545288439702e-15
first50_flexibility
9.757628167637071e-14
first50_hydrophilicity
-4.2876310514789315e-16
first50_surface_accessibility
-3.3851126086531744e-14
first50_janin
7.09065299266811e-16
last50_hydrophobicity
-9.406550022003535e-17
last50_flexibility
2.1100801331999818e-13
last50_hydrophilicity
5.40202277658

# Models

In [11]:
table = collections.defaultdict(list)
columns = ['params', 
           'mean_test_Accuracy', 'std_test_Accuracy', 
           'mean_test_Precision', 'std_test_Precision',
           'mean_test_Recall', 'std_test_Recall',
           'mean_test_F1', 'std_test_F1',
           'mean_test_MCC', 'std_test_MCC']

### Test using Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Define some hyperparameters we are going to test
param_dist = {"n_estimators": [1000]}

# Make a handle for our model (initialization or our model without fitted parameters)
rf = RandomForestClassifier(random_state=79)

rf_random_search_res = cross_validate(rf,
                                      full_x,
                                      full_y,
                                      param_dist,
                                      5)

print("Best performance: %s" % (rf_random_search_res.best_score_))
print("With the hyperparameters: %s" % (rf_random_search_res.best_params_))

# Print cross-validation results
rf_df = pd.DataFrame.from_dict(rf_random_search_res.cv_results_)[columns].round(4)
rf_result = rf_df[rf_df['params']==rf_random_search_res.best_params_].reset_index(drop=True).to_dict('records')[0]
table = make_row(table, rf_result, "RF")
rf_df.T

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] n_estimators=1000 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=1000, Accuracy=0.674, F1=0.688, MCC=0.541, Precision=0.702, Recall=0.679, total=  40.8s
[CV] n_estimators=1000 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   40.8s remaining:    0.0s


[CV]  n_estimators=1000, Accuracy=0.680, F1=0.691, MCC=0.551, Precision=0.701, Recall=0.686, total=  40.3s
[CV] n_estimators=1000 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


[CV]  n_estimators=1000, Accuracy=0.654, F1=0.674, MCC=0.516, Precision=0.676, Recall=0.672, total=  40.3s
[CV] n_estimators=1000 ...............................................
[CV]  n_estimators=1000, Accuracy=0.662, F1=0.687, MCC=0.525, Precision=0.699, Recall=0.678, total=  41.1s
[CV] n_estimators=1000 ...............................................
[CV]  n_estimators=1000, Accuracy=0.667, F1=0.687, MCC=0.533, Precision=0.694, Recall=0.682, total=  40.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.4min finished


Best performance: 0.6672556598564329
With the hyperparameters: {'n_estimators': 1000}


,0
params,{'n_estimators': 1000}
mean_test_Accuracy,0.6673
std_test_Accuracy,0.009
mean_test_Precision,0.6943
std_test_Precision,0.0096
mean_test_Recall,0.6794
std_test_Recall,0.0045
mean_test_F1,0.6854
std_test_F1,0.006
mean_test_MCC,0.5332


### Test using MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

# Define some hyperparameters we are going to test
param_dist = {"hidden_layer_sizes": [100,128,256,512]}

# Make a handle for our model (initialization or our model without fitted parameters)
mlp = MLPClassifier(random_state=79)

mlp_random_search_res = cross_validate(mlp,
                                       full_x,
                                       full_y,
                                       param_dist,
                                       5)

print("Best pemlpormance: %s" % (mlp_random_search_res.best_score_))
print("With the hyperparameters: %s" % (mlp_random_search_res.best_params_))

# Print cross-validation results
mlp_df = pd.DataFrame.from_dict(mlp_random_search_res.cv_results_)[columns].round(4)
mlp_result = mlp_df[mlp_df['params']==mlp_random_search_res.best_params_].reset_index(drop=True).to_dict('records')[0]
table = make_row(table, mlp_result, "MLP")
mlp_df.T

### Test using SVM

In [ ]:
from sklearn.svm import SVC

# Define some hyperparameters we are going to test
param_dist = {"C": [1,2,3,5,10]}

# Make a handle for our model (initialization or our model without fitted parameters)
svm = SVC(random_state=79)

svm_random_search_res = cross_validate(svm,
                                       full_x,
                                       full_y,
                                       param_dist,
                                       5)

print("Best performance: %s" % (svm_random_search_res.best_score_))
print("With the hyperparameters: %s" % (svm_random_search_res.best_params_))

# Print cross-validation results
svm_df = pd.DataFrame.from_dict(svm_random_search_res.cv_results_)[columns].round(4)
svm_result = svm_df[svm_df['params']==svm_random_search_res.best_params_].reset_index(drop=True).to_dict('records')[0]
table = make_row(table, svm_result, "SVM")
svm_df.T

### Test using GNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
# Define some hyperparameters we are going to test
param_dist = {"priors":[None]}

# Make a handle for our model (initialization or our model without fitted parameters)
gnb = GaussianNB()

gnb_random_search_res = cross_validate(gnb,
                                       full_x,
                                       full_y,
                                       param_dist,
                                       5)

print("Best pegnbormance: %s" % (gnb_random_search_res.best_score_))
print("With the hyperparameters: %s" % (gnb_random_search_res.best_params_))

# Print cross-validation results
gnb_df = pd.DataFrame.from_dict(gnb_random_search_res.cv_results_)[columns].round(4)
gnb_result = gnb_df[gnb_df['params']==gnb_random_search_res.best_params_].reset_index(drop=True).to_dict('records')[0]
table = make_row(table, gnb_result, "GNB")
gnb_df.T

### Test using LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Define some hyperparameters we are going to test
param_dist = {"priors":[None]}

# Make a handle for our model (initialization or our model without fitted parameters)
lda = LinearDiscriminantAnalysis()

lda_random_search_res = cross_validate(lda,
                                       full_x,
                                       full_y,
                                       param_dist,
                                       5)

print("Best peldaormance: %s" % (lda_random_search_res.best_score_))
print("With the hyperparameters: %s" % (lda_random_search_res.best_params_))

# Print cross-validation results
lda_df = pd.DataFrame.from_dict(lda_random_search_res.cv_results_)[columns].round(4)
lda_result = lda_df[lda_df['params']==lda_random_search_res.best_params_].reset_index(drop=True).to_dict('records')[0]
table = make_row(table, lda_result, "LDA")
lda_df.T

### Test using KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Define some hyperparameters we are going to test
param_dist = {"n_neighbors":[1,2,3,4,5,6,7,8,9,10,15,20,25,30]}

# Make a handle for our model (initialization or our model without fitted parameters)
knn = KNeighborsClassifier()

knn_random_search_res = cross_validate(knn,
                                       full_x,
                                       full_y,
                                       param_dist,
                                       5)

print("Best peknnormance: %s" % (knn_random_search_res.best_score_))
print("With the hyperparameters: %s" % (knn_random_search_res.best_params_))

# Print cross-validation results
knn_df = pd.DataFrame.from_dict(knn_random_search_res.cv_results_)[columns].round(4)
knn_result = knn_df[knn_df['params']==knn_random_search_res.best_params_].reset_index(drop=True).to_dict('records')[0]
table = make_row(table, knn_result, "KNN")
knn_df.T

### Get Table

In [ ]:
table = pd.DataFrame.from_dict(table)
table

In [ ]:
print(table.to_latex())

### Get Confusion Matrix and Classification Report and Feature Importance

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=79)
mean_conf_matrix, std_conf_matrix, class_reports = get_conf_matrix(rf, 
                                                                   full_x.to_numpy(), 
                                                                   full_y, classes, 
                                                                   n_splits=5)
# Get labels for Confusion matrix plot
mean_str = mean_conf_matrix.astype(str)
std_str = std_conf_matrix.astype(str)
labels = np.core.defchararray.add(mean_str,"±\n")
labels = np.core.defchararray.add(labels,std_str)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=79)
rf.fit(full_x, full_y)

In [ ]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Plot the feature importances of the forest
plt.figure(figsize=(30,10))
plt.title("Feature importances")
plt.bar(range(train_x.shape[1]), 
        importances[indices], 
        color="r", yerr=std[indices], align="center")
plt.xticks(range(train_x.shape[1]), indices)
plt.xlim([-1, train_x.shape[1]])
plt.xlabel("Features")
plt.ylabel("Importance")
plt.grid()
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 24})
# Plot averaged confusion matrix
df_cm = pd.DataFrame(mean_conf_matrix, 
                     index = classes,
                     columns = classes)
plt.figure(figsize = (16,12))
sns.heatmap(df_cm, annot=labels, fmt='', cmap='Blues',
            cbar=True, square=True, center=15,
            linewidths=.1, linecolor='black')
plt.ylim((4,0))
plt.title("Confusion Matrix \n Mean Confusion Error (Counts) ± Std")
# plt.title("Normalised Confusion Matrix \n Mean Proportion of True Labels Predicted to be in each class ± Std")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.savefig("confusion_matrix.pdf", format="pdf")
plt.show()

In [ ]:
class_reports = np.array(class_reports)
pd.DataFrame(class_reports[:3,:]).T

In [ ]:
print(pd.DataFrame(class_reports[:3,:], columns=classes).T.to_latex())

### Test of blind test data

In [ ]:
class_labels = ['Cyto', 'Secr', 'Mito', 'Nucl']
pred = rf.predict_proba(blind_test_x )
seq_id = [seq.id for seq in blind_test_xs]

In [ ]:
blind_test_results = collections.defaultdict(list)
for i, seq in enumerate(seq_id):
    blind_test_results[seq].extend([class_labels[np.argmax(pred[i,:])], str(int((np.max(pred[i,:])*100).round(0)))])
pd.DataFrame.from_dict(blind_test_results).T

In [ ]:
print(pd.DataFrame.from_dict(blind_test_results).T.to_latex())

In [ ]:
# rf = SVC(C=5,random_state=79,probability=True)
rf = RandomForestClassifier(n_estimators=1000, random_state=79)
rf.fit(train_x, train_y)

In [ ]:
pred_proba_test = rf.predict_proba(test_x)
pred_proba_test_label = np.argmax(pred_proba_test,axis=1)
pred_proba_test_conf = np.max(pred_proba_test,axis=1)

In [ ]:
pred_proba_test_conf[test_y!=pred_proba_test_label].mean()

In [ ]:
pred_proba_test_conf[test_y==pred_proba_test_label].mean()